In [1]:
# Install the plaidml backend
# import plaidml.keras
# plaidml.keras.install_backend()

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from keras import backend as K
from keras import models
from keras.models import model_from_json, Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# %matplotlib notebook
# K.set_learning_phase(0)

/home/pingo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Lendo modelos da lista de melhores

# carregando Base de Dados e Fazendo janelamento 

### uma janela de tamanho ordem tem (tamanho da base - ordem) janelas 

In [2]:
caminhoInstancia = 'Instancia/EnaSudNovoAte2016.csv'
BaseDados = np.loadtxt( caminhoInstancia, 'float', delimiter=',')
BaseDadosOriginal = BaseDados
# print(BaseDados.ravel())
# BaseDados = MinMaxScaler().fit_transform(BaseDados)
# NORMALIZANDO
BaseDados = BaseDados.ravel()
BaseDados = BaseDados.reshape(len(BaseDados), 1)
scaler = StandardScaler().fit(BaseDados)
BaseDados = scaler.transform(BaseDados)
maxSerie = max(BaseDados)
minSerie = min(BaseDados)
BaseDados = (BaseDados - minSerie) / (maxSerie - minSerie)
# print(BaseDados)
# tamanhoBase = len(BaseDados)
# pd.DataFrame(BaseDados)
nomesMeses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# representações da Base de Dados
+ caracterizações da Base de Dados EnaSud

In [3]:
# ordensModelos = [modelo.input_shape[1] for modelo in modelos]

In [4]:
def janelamento(serie, ordem):
    """
    Recebe uma serie e retorna um ndArray com n linhas janeladas de ordem em ordem
    """    
    tamanhoSerie = len(serie)
#     print(tamanhoSerie)
    yAtual = ordem
    x = list()
    y = list()
    while(yAtual < tamanhoSerie):
        x.append(serie[(yAtual-ordem):yAtual])
        y.append(serie[yAtual])
        yAtual+=1
    data = pd.DataFrame(x)
    data['Y'] = y
    return data

In [5]:

# Faz o janelamento em todos os meses Separando os meses
BaseComJanelamento = janelamento(BaseDados.ravel(), 12) #{nomeMes:baseComJanelamento for nomeMes, baseComJanelamento in zip(nomesMeses, [janelamento(BaseDados[:, x], ordensModelos[x]) for x in range(12)])}
# BaseComJanelamento

# divisao em  treino, val e teste

In [6]:
tamanhoVal = 5*12
tamanhoTeste = 5*12
# tamanhoTreino = BaseDadosOriginal.shape()# - (tamanhoVal + tamanhoTeste)
# print(len(BaseDadosOriginal)-(tamanhoVal + tamanhoTeste))
# treino = BaseDados[:tamanhoTreino]
# val = BaseDados[tamanhoTreino: tamanhoTreino+tamanhoVal]
# teste = BaseDados[tamanhoTreino+tamanhoVal:]
# dicionário de dataFrames
treino = BaseComJanelamento[:-(tamanhoTeste)]#BaseComJanelamento[:-(tamanhoVal+tamanhoTeste)]
# val = BaseComJanelamento[-(tamanhoVal+tamanhoTeste):-(tamanhoTeste)]
teste = BaseComJanelamento[-(tamanhoTeste):]
# [teste['Dezembro'] for mes in nomesMeses]

# treino Modelo

In [7]:
ordem = 12
# tamVal = 5
numNeuronios = 50
numEpocas = 1000
treinoX = np.array(treino.iloc[:, :-1])
treinoY = np.array(treino.iloc[:, -1])
testeX = np.array(teste.iloc[:, :-1])
testeY = np.array(teste.iloc[:, -1])
treinoX = np.reshape(treinoX, (treinoX.shape[0], 1, treinoX.shape[1]))
testeX = np.reshape(testeX, (testeX.shape[0], 1, testeX.shape[1]))
# testeX = np.reshape(testeX, (testeX.shape[0], 1, testeX.shape[1]))
# treinoX, treinoY

In [ ]:
modelo = Sequential()
modelo.add(LSTM(numNeuronios, return_sequences=False, input_shape=(1, ordem )))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(500, return_sequences=False, input_shape=(1, ordem)))

# modelo.add(LSTM(250, return_sequences=True, input_shape=(1, ordem)))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(100, return_sequences=True, input_shape=(1, ordem)))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(50, return_sequences=True, input_shape=(1, ordem)))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(25, return_sequences=True, input_shape=(1, ordem)))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(10, return_sequences=False, input_shape=(1, ordem)))
# modelo.add(Dropout(0.5))
# modelo.add(LSTM(numNeuronios))
# modelo.add(Dropout(0.8))
# model.add(LSTM(4, input_shape=(1, look_back)))
modelo.add(Dense(1))
modelo.compile(loss='mean_absolute_error', optimizer='adam',  metrics=['mean_squared_error', 'mean_absolute_percentage_error'])
salvaModelo = ModelCheckpoint('modeloLstm'+'.hdf5', monitor='val_mean_absolute_percentage_error', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlyStopping = EarlyStopping(monitor='val_loss', patience=300, verbose=0, mode='auto')
hist = modelo.fit(treinoX, treinoY, validation_split=(tamanhoVal/len(treinoY)), batch_size=1, epochs=numEpocas, verbose=2, callbacks=[salvaModelo])

Train on 900 samples, validate on 60 samples
Epoch 1/1000
5s - loss: 0.0684 - mean_squared_error: 0.0099 - mean_absolute_percentage_error: 62532.6908 - val_loss: 0.0735 - val_mean_squared_error: 0.0101 - val_mean_absolute_percentage_error: 34.2901
Epoch 2/1000
3s - loss: 0.0569 - mean_squared_error: 0.0064 - mean_absolute_percentage_error: 66369.6186 - val_loss: 0.0651 - val_mean_squared_error: 0.0086 - val_mean_absolute_percentage_error: 33.8563
Epoch 3/1000
4s - loss: 0.0543 - mean_squared_error: 0.0059 - mean_absolute_percentage_error: 68190.9679 - val_loss: 0.0676 - val_mean_squared_error: 0.0089 - val_mean_absolute_percentage_error: 33.5174
Epoch 4/1000
3s - loss: 0.0540 - mean_squared_error: 0.0057 - mean_absolute_percentage_error: 49005.5450 - val_loss: 0.0631 - val_mean_squared_error: 0.0082 - val_mean_absolute_percentage_error: 28.3555
Epoch 5/1000
3s - loss: 0.0524 - mean_squared_error: 0.0053 - mean_absolute_percentage_error: 70195.1881 - val_loss: 0.0636 - val_mean_squared_

In [ ]:
modelo = models.load_model('modeloLstm.hdf5')

# graficos metricas treino

In [ ]:
plt.figure(figsize=[16,9])
plt.subplot(121)
plt.plot(hist.history['val_loss'], label='MAE Validacao')
plt.plot(hist.history['loss'], label='MAE Treino')
plt.legend()
plt.grid(b=True, axis='y')
plt.subplot(122)
plt.plot(hist.history['val_mean_absolute_percentage_error'], label='MAPE Validacao')
# plt.plot(hist.history['mean_absolute_percentage_error'], label='MAE Treino')
plt.grid(b=True, axis='y')

plt.legend()
# plt.close()

# histograma de Mapes por mes fazer depois

In [ ]:
plt.figure(figsize=[16,9])
previsao = modelo.predict(testeX)
x = np.arange(len(previsao))
plt.plot(x, testeY, label='real', marker='o')
plt.plot(x, previsao, label='previsao', marker='o')
plt.xticks(x, [nomesMeses[x][0] for x in range(12)]*5)
plt.grid(b=True, axis='x')
plt.title('LSTM com uma camada oculta')
plt.xlabel('meses em 5 anos')
plt.ylabel('MAE')
plt.legend()
# plt.close()


In [ ]:
modelo = models.load_model('modeloLstm2.hdf5')

In [ ]:
plt.figure(figsize=[16,9])
plt.subplot(121)
plt.plot(hist.history['val_loss'], label='MAE Validacao')
plt.plot(hist.history['loss'], label='MAE Treino')
plt.legend()
plt.grid(b=True, axis='y')
plt.subplot(122)
plt.plot(hist.history['val_mean_absolute_percentage_error'], label='MAPE Validacao')
# plt.plot(hist.history['mean_absolute_percentage_error'], label='MAE Treino')
plt.grid(b=True, axis='y')

plt.legend()
# plt.close()

In [ ]:
plt.figure(figsize=[16,9])
previsao = modelo.predict(testeX)
x = np.arange(len(previsao))
plt.plot(x, testeY, label='real', marker='o')
plt.plot(x, previsao, label='previsao', marker='o')

plt.xticks(x, [nomesMeses[x][0] for x in range(12)]*5)
plt.grid(b=True, axis='x')
plt.title('LSTM com uma camada oculta')
plt.xlabel('meses em 5 anos')
plt.ylabel('MAE')
plt.legend()
# plt.close()

In [ ]:
# plt.figure(figsize=[16,9])
plt.bar(np.arange(1,13), [modelos[nomesMeses.index(mes)].evaluate((teste[mes]).iloc[:, :-1], (teste[mes]).iloc[:, -1], verbose=0)[0] for mes in nomesMeses])
plt.xticks(np.arange(1,13), nomesMeses, rotation=90)
plt.xlabel('Meses ')
plt.ylabel('Mapes (%)')
plt.grid(b='on', axis='y')
plt.ylim(0, 50)
plt.title('Mapes encontrados para cada Mês utilizando os cenários como teste')
plt.savefig('graficos/MapesTestePorMes.jpg', dpi=300)

In [ ]:
# [modelos[nomesMeses.index(mes)].predict((teste[mes]).iloc[:, :-1]) for mes in nomesMeses]

# Pegar os Resíduos do treino

In [ ]:
previsoesModelos_treino = {mes:modelos[nomesMeses.index(mes)].predict((treino[mes]).iloc[:, :-1]).ravel() for mes in nomesMeses}

In [ ]:
residuos_previsaoxreal = { mes:previsoesModelos_treino[mes] - treino[mes]['Y'] for mes in nomesMeses}

In [ ]:
# residuos_previsaoxreal

# Encaixar Resíduos em uma distribuição
+ verificar qual a melhor distribuição (p-Values KS)
+ listar melhores P-Values

In [ ]:
distribuicoes = [ 'alpha', 'anglit', 'arcsine', 'beta', 'betaprime', 'bradford' , 'burr', 'cauchy', 'chi', 'chi2', 'cosine', 'dgamma', 'dweibull', 'erlang', 'expon', 'exponweib', 'exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy', 'halflogistic', 'halfnorm', 'hypsecant', 'invgamma', 'invgauss', 'invweibull', 'johnsonsb', 'johnsonsu', 'ksone', 'kstwobign', 'laplace', 'logistic', 'loggamma', 'loglaplace', 'lognorm', 'lomax', 'maxwell', 'mielke', 'nakagami', 'ncx2', 'ncf', 'nct', 'norm', 'pareto', 'pearson3', 'powerlaw', 'powerlognorm', 'powernorm', 'rdist', 'reciprocal', 'rayleigh', 'rice', 'recipinvgauss', 'semicircular', 't', 'triang', 'truncexpon', 'truncnorm', 'tukeylambda', 'uniform', 'vonmises', 'wald', 'weibull_min', 'weibull_max', 'wrapcauchy']

In [ ]:
 # warnings.filterwarnings('DeprecationWarn' ,category=RuntimeWarning) 

mes_dist_ks = dict()
for mes, residuosVal in residuos_previsaoxreal.items():
#     print(mes)
    KSsDist = list()
    for distribuicao in distribuicoes:
#         print(distribuicao)                
        dist = getattr(scipy.stats, distribuicao)
        parametros = dist.fit(residuosVal)
        ks = scipy.stats.kstest(residuosVal, distribuicao, parametros)
        KSsDist.append((dist, parametros, ks[1]))
    KSsDist = pd.DataFrame(KSsDist)
    melhorLinha = KSsDist[2].max()      
#     print(KSsDist.iat(pd.Series(melhorLinha)))
#     print(KSsDist.loc(KSsDist[2]==melhorLinha, '2'))    
    mes_dist_ks[mes] = KSsDist.loc[KSsDist[2] == melhorLinha]
# print(mes_dist_ks)
# mes_dist_ks = pd.DataFrame(mes_dist_ks)
# print(mes_dist_ks)            

In [ ]:
# listPvalues = [mes_dist_ks[mes][2] for mes in nomesMeses]
# print(list(listPvalues))
pValues_mes = { mes: pValue.values   for mes, pValue in zip(nomesMeses, [mes_dist_ks[mes] for mes in nomesMeses])}

In [ ]:
pValues_mes#, index=['pValues']).T

In [ ]:
[pValues_mes[mes][0][2] for mes in nomesMeses]

In [ ]:
plt.figure(figsize=[16,9])
plt.bar(np.arange(1,13),[pValues_mes[mes][0][2] for mes in nomesMeses])
plt.xticks(np.arange(1,13), [mes+'\n'+pValues_mes[mes][0][0].name for mes in nomesMeses])
plt.xlabel('P-Values')
plt.ylabel('meses')
plt.grid(b='on', axis='y')
plt.title('P-Values dos resíduos em relação à Melhor Distribuição')
plt.savefig('graficos/PvaluesDistribuicaoResiduos.jpg')
plt.close()

# Observações:
## <span style="color:blue"> Os p-Values encontrados mostram que os resíduos foram muito bem descritos pelas distribuicões teóricas utilizadas.</span>

# gerar cenarios e plotar
+ todos os cenários tem o mesmo tamanho + ordem

In [ ]:
# pega lambdas e soma nos cenários
numCenarios = 1000
cenarios = {mes:(mes_dist_ks[mes][0].iloc[0](*mes_dist_ks[mes][1].iloc[0])).rvs(size=(numCenarios + modelos[nomesMeses.index(mes)].input_shape[1]) ) + treino[mes]['Y'].iloc[-1] for mes in nomesMeses }
# cenarios

In [ ]:
mediasAfluenciaAnosTreino = { mes : treino[mes]['Y'].iloc[:].mean() for mes in nomesMeses }


# previsão somente com distribuicoes ( monte carlo )

In [ ]:
# x = np.arange(1,13)
# plt.xlim(1,12)
# # plt.tight_layout()
# # plt.figure(figsize=(16,9))

# plt.plot(x, mediasAfluenciaAnosTreino.values())
# plt.plot(x, pd.DataFrame(cenarios).mean(axis=0))    
# plt.plot(np.transpose(x).reshape(12, 1), cenarios.T, c='black', linewidth=0.1)


# cenarios['Janeiro'].mean()

# Aplicando modelos nos resíduos
+ Janelamento cenários
+ Aplicar modelos mês a mês 


In [ ]:
cenariosComJanelamento = {mes: janelamento(cenarios[mes], modelos[nomesMeses.index(mes)].input_shape[1]) for mes in nomesMeses }
# cenariosComJanelamento['Janeiro']

## Mape quando aplicado ao modelo

In [ ]:
plt.figure(figsize=[16,9])
plt.bar(np.arange(1,13),[ modelos[nomesMeses.index(mes)].evaluate((cenariosComJanelamento[mes]).iloc[:, :-1], (cenariosComJanelamento[mes]).iloc[:, -1], verbose=0)[0] for mes in nomesMeses ])
plt.xticks(np.arange(1,13), nomesMeses)
plt.grid(b='on', axis='y')
plt.xlabel('Meses (%)')
plt.ylabel('Mapes')
plt.title('Mapes encontrados para cada Mês utilizando os cenários como teste')
plt.close()

In [ ]:
previsoesNosCenarios = {mes:modelos[nomesMeses.index(mes)].predict((cenariosComJanelamento[mes]).iloc[:, :-1]).reshape(5, int(numCenarios/5)) for mes in nomesMeses}
# previsoesNosCenarios['Janeiro']

In [ ]:
mediaPrevisoesNosCenarios = [np.mean(previsoesNosCenarios[mes], axis=1) for mes in nomesMeses]
stdPrevisoesNosCenarios = [np.std(previsoesNosCenarios[mes], axis=1) for mes in nomesMeses]
stdPrevisoesNosCenarios = np.array(stdPrevisoesNosCenarios).T.ravel()
mediaPrevisoesNosCenarios = np.array(mediaPrevisoesNosCenarios).T.ravel()
# stdPrevisoesNosCenarios
# pd.DataFrame(mediaPrevisoesNosCenarios)


In [ ]:
mediaConjuntoTeste = [teste[mes]['Y'].mean() for mes in nomesMeses]
# plt.subplot(2,1,1)
# plt.figure(figsize=[16,9])
# # boxplot dos meses para cenário
# pd.DataFrame([teste[mes]['Y'] for mes in nomesMeses]).T.boxplot()
# plt.subplot(2,1,2)
# pd.DataFrame(BaseDadosOriginal).boxplot()
# # plt.bar(np.arange(1,13),[ modelos[nomesMeses.index(mes)].evaluate((cenariosComJanelamento[mes]).iloc[:, :-1], (cenariosComJanelamento[mes]).iloc[:, -1], verbose=0)[0] for mes in nomesMeses ])
# plt.xticks(np.arange(1,13), nomesMeses)
# plt.xlabel('Meses')
# plt.ylabel('Afluência Normalizada entre [0,1]')
# plt.title('BoxPlot dos cenários gerados')



In [ ]:
plt.figure(figsize=(20, 9))
x = np.arange(1, 61, 1)
plt.plot(x, mediaConjuntoTeste*5, label='media Conjunto de Teste', marker='o')
plt.plot(x, mediaPrevisoesNosCenarios, label='media Previsões', marker='o')
# plt.plot(x, np.reshape(previsoesNosCenariosAux, (12, numCenarios)), c='black', linewidth=0.1 )
# plt.subplot((1,2,2))
plt.ylabel('Afluência Normalizada')
plt.xlabel('Meses (5 Anos)')
plt.xlim(1, 60)
plt.title('Previsão em vários cenários EnaSud')
plt.legend()
nomesMeses_3 = [mes[0:3] for mes in nomesMeses]
plt.xticks(x, nomesMeses_3 *5, rotation=90)
plt.show()
plt.close()

In [ ]:
# plotar as linhas dos cenários 
previsoesNosCenariosAux = [previsoesNosCenarios[mes] for mes in nomesMeses]
# arrumar para plotar 
previsoesNosCenariosAux = np.transpose(previsoesNosCenariosAux, (1, 0, 2))
previsoesNosCenariosAux = np.reshape(previsoesNosCenariosAux, (12*5, int(numCenarios/5)))

In [ ]:
plt.figure(figsize=(20, 9))
x = np.arange(1, 61, 1)
plt.plot(x, previsoesNosCenariosAux, c='black', linewidth=0.1 )
plt.plot(x, mediaConjuntoTeste*5, label='media Conjunto de Teste')
plt.plot(x, mediaPrevisoesNosCenarios, label='media Previsões')
# plt.subplot((1,2,2))
plt.ylabel('Afluência Normalizada')
plt.xlabel('Meses')
plt.xlim(1, 60)
plt.title('Previsão em vários cenários EnaSud')
plt.legend()
plt.xticks(x, nomesMeses_3*5, rotation=90 )
plt.savefig('graficos/PrevisaoCenarios.jpg')
plt.close()

In [ ]:
plt.figure(figsize=(20, 9))
x = np.arange(1, 61, 1)
plt.plot(x, mediaConjuntoTeste*5, label='media Conjunto de Teste', marker='o')
plt.errorbar(x, mediaPrevisoesNosCenarios, yerr=stdPrevisoesNosCenarios, label='media Previsões nos cenários', marker='o', elinewidth=0.3, ecolor='black', mfc='red', color='orange', capsize=5)
# plt.plot(x, np.reshape(previsoesNosCenariosAux, (12, numCenarios)), c='black', linewidth=0.1 )
# plt.subplot((1,2,2))
plt.ylabel('Afluência Normalizada')
plt.xlabel('Meses')
plt.xlim(1, 60)
plt.title('Previsão em vários cenários EnaSud')
plt.legend()
nomesMeses_3 = [mes[0:3] for mes in nomesMeses]
plt.xticks(x, nomesMeses_3 *5, rotation=90)
plt.savefig('graficos/PrevisaoCenariosVariacao.jpg')
plt.close()